In [13]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import os

# Satır sonu ve boşlukları temizlemek için
def temizle_metin(text):
    if not isinstance(text, str):
        return text
    return text.replace("\n", " ").replace("\r", " ").strip()

# CAPTCHA için manuel bekleme fonksiyonu
def captcha_bekle(mod, index):
    if index % 10 == 0:
        input(f"🛑 CAPTCHA varsa ({mod}) çöz ve Enter'a bas...")

# 📌 CSV'yi sıfırlamak için:
csv_dosya_adi = "istanbulkiralik.csv"
if os.path.exists(csv_dosya_adi):
    os.remove(csv_dosya_adi)

# Kullanıcıdan txt dosyası al
txt_file = input("txt dosyası: ")

# Chrome başlat (uc ile)
driver = uc.Chrome()

# txt dosyasındaki her satır için işlemleri yap
with open(txt_file, "r", encoding="utf-8") as file:
    for line in file:
        ilanlar_listesi = []
        line = line.strip().split(",")
        excel_adi = line[0]
        baslangic = int(line[1])
        bitis = int(line[2])
        base_url = line[3]

        print(f"\n⏳ {excel_adi} için sayfalar geziliyor...")

        ilan_linkleri = []
        for sayfa in range(baslangic, bitis + 1):
            captcha_bekle("SAYFA", sayfa)
            sayfa_url = f"{base_url}?page={sayfa}"
            print(f"🔎 Sayfa {sayfa}: {sayfa_url}")
            driver.get(sayfa_url)

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.card-link"))
                )
                kartlar = driver.find_elements(By.CSS_SELECTOR, "a.card-link")
                linkler = [k.get_attribute("href") for k in kartlar]
                ilan_linkleri.extend(linkler)
                print(f"✅ {len(linkler)} ilan bulundu.")
            except:
                print("❌ Sayfa yüklenemedi veya ilan bulunamadı.")

            time.sleep(random.uniform(2.5, 5.0))

        print(f"\n📦 Toplam {len(ilan_linkleri)} ilan linki bulundu. Veriler çekiliyor...")

        for i, ilan_url in enumerate(ilan_linkleri, 1):
            captcha_bekle("İLAN", i)
            print(f"\n📄 {i}. ilan işleniyor: {ilan_url}")
            try:
                driver.get(ilan_url)
            except:
                print("⚠️ İlan sayfasına gidilemedi, atlanıyor.")
                continue

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "h1"))
                )
            except:
                print("❌ İlan sayfası yüklenmedi.")
                continue

            try:
                baslik = driver.find_element(By.TAG_NAME, "h1").text.strip()
            except:
                baslik = "-"

            try:
                fiyat = driver.find_element(By.CSS_SELECTOR, "p.price").text.strip()
            except:
                fiyat = "-"

            # Konum bilgileri
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "ul.detail-info-location li"))
                )
                konum = driver.find_elements(By.CSS_SELECTOR, "ul.detail-info-location li")
                sehir = konum[0].text.strip() if len(konum) > 0 else "-"
                ilce = konum[1].text.strip() if len(konum) > 1 else "-"
                mahalle = konum[2].text.strip() if len(konum) > 2 else "-"
            except:
                sehir, ilce, mahalle = "-", "-", "-"

            # Teknik bilgiler
            konut_tipi = oda_sayisi = brut_net = banyo_sayisi = kat = kat_sayisi = bina_yasi = isinma = esya = yapi = yapi_durumu = aidat = depozito = yakit = cephe = "-"

            try:
                teknik_liste = driver.find_elements(By.CSS_SELECTOR, "ul.adv-info-list li.spec-item")
                for item in teknik_liste:
                    text = item.text.strip()
                    if "Konut Tipi" in text:
                        konut_tipi = text.replace("Konut Tipi", "").strip()
                    elif "Oda Sayısı" in text:
                        oda_sayisi = text.replace("Oda Sayısı", "").strip()
                    elif "Brüt / Net M2" in text:
                        brut_net = text.replace("Brüt / Net M2", "").strip()
                    elif "Banyo Sayısı" in text:
                        banyo_sayisi = text.replace("Banyo Sayısı", "").strip()
                    elif "Bulunduğu Kat" in text:
                        kat = text.replace("Bulunduğu Kat", "").strip()
                    elif "Kat Sayısı" in text:
                        kat_sayisi = text.replace("Kat Sayısı", "").strip()
                    elif "Bina Yaşı" in text:
                        bina_yasi = text.replace("Bina Yaşı", "").strip()
                    elif "Isınma Tipi" in text:
                        isinma = text.replace("Isınma Tipi", "").strip()
                    elif "Eşya Durumu" in text:
                        esya = text.replace("Eşya Durumu", "").strip()
                    elif "Yapı Tipi" in text:
                        yapi = text.replace("Yapı Tipi", "").strip()
                    elif "Yapının Durumu" in text:
                        yapi_durumu = text.replace("Yapının Durumu", "").strip()
                    elif "Aidat" in text:
                        aidat = text.replace("Aidat", "").strip()
                    elif "Depozito" in text:
                        depozito = text.replace("Depozito", "").strip()
                    elif "Yakıt Tipi" in text:
                        yakit = text.replace("Yakıt Tipi", "").strip()
                    elif "Cephe" in text:
                        cephe = text.replace("Cephe", "").strip()
            except:
                print("⚠️ Teknik bilgiler alınamadı.")

            ilanlar_listesi.append({
                "Başlık": baslik,
                "Fiyat": fiyat,
                "Şehir": sehir,
                "İlçe": ilce,
                "Mahalle": mahalle,
                "Konut Tipi": konut_tipi,
                "Oda Sayısı": oda_sayisi,
                "Brüt / Net M2": brut_net,
                "Banyo Sayısı": banyo_sayisi,
                "Kat": kat,
                "Kat Sayısı": kat_sayisi,
                "Bina Yaşı": bina_yasi,
                "Isınma Tipi": isinma,
                "Eşya Durumu": esya,
                "Yapı Tipi": yapi,
                "Yapının Durumu": yapi_durumu,
                "Aidat": aidat,
                "Depozito": depozito,
                "Yakıt Tipi": yakit,
                "Cephe": cephe,
                "Link": ilan_url
            })

            time.sleep(random.uniform(2.5, 5.0))

        # CSV'ye append et
        df = pd.DataFrame(ilanlar_listesi)
        df = df.applymap(temizle_metin)
        write_header = not os.path.exists(csv_dosya_adi)
        df.to_csv(csv_dosya_adi, mode='a', index=False, header=write_header, encoding="utf-8-sig")
        print(f"\n📥 Veriler eklendi: {csv_dosya_adi}")

driver.quit()
print("\n🎉 Tüm işlemler tamamlandı.")


txt dosyası:  linkler.txt



⏳ istanbulkiralik_bolum1 için sayfalar geziliyor...
🔎 Sayfa 1: https://www.hepsiemlak.com/istanbul-kiralik?page=1
✅ 24 ilan bulundu.
🔎 Sayfa 2: https://www.hepsiemlak.com/istanbul-kiralik?page=2
✅ 24 ilan bulundu.
🔎 Sayfa 3: https://www.hepsiemlak.com/istanbul-kiralik?page=3
✅ 24 ilan bulundu.
🔎 Sayfa 4: https://www.hepsiemlak.com/istanbul-kiralik?page=4
✅ 24 ilan bulundu.
🔎 Sayfa 5: https://www.hepsiemlak.com/istanbul-kiralik?page=5
✅ 24 ilan bulundu.
🔎 Sayfa 6: https://www.hepsiemlak.com/istanbul-kiralik?page=6
✅ 24 ilan bulundu.
🔎 Sayfa 7: https://www.hepsiemlak.com/istanbul-kiralik?page=7
✅ 24 ilan bulundu.
🔎 Sayfa 8: https://www.hepsiemlak.com/istanbul-kiralik?page=8
✅ 24 ilan bulundu.
🔎 Sayfa 9: https://www.hepsiemlak.com/istanbul-kiralik?page=9
✅ 24 ilan bulundu.


🛑 CAPTCHA varsa (SAYFA) çöz ve Enter'a bas... 


🔎 Sayfa 10: https://www.hepsiemlak.com/istanbul-kiralik?page=10
✅ 24 ilan bulundu.

📦 Toplam 240 ilan linki bulundu. Veriler çekiliyor...

📄 1. ilan işleniyor: https://www.hepsiemlak.com/istanbul-esenyurt-barbaros-hayrettin-pasa-kiralik/daire/142108-121

📄 2. ilan işleniyor: https://www.hepsiemlak.com/istanbul-esenyurt-mehtercesme-kiralik/daire/142108-104

📄 3. ilan işleniyor: https://www.hepsiemlak.com/istanbul-esenyurt-yesilkent-kiralik/daire/142108-97

📄 4. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sariyer-rumeli-kavagi-kiralik/daire/11217-9277

📄 5. ilan işleniyor: https://www.hepsiemlak.com/istanbul-eyupsultan-mimar-sinan-kiralik/daire/33208-5819

📄 6. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sariyer-maden-kiralik/daire/33208-5857

📄 7. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sariyer-uskumrukoy-kiralik/villa/33208-5852

📄 8. ilan işleniyor: https://www.hepsiemlak.com/istanbul-besiktas-bebek-kiralik/daire/25444-7048

📄 9. ilan işleniyor: https://www

🛑 CAPTCHA varsa (İLAN) çöz ve Enter'a bas... 


...

🛑 CAPTCHA varsa (İLAN) çöz ve Enter'a bas... 



📄 250. ilan işleniyor: https://www.hepsiemlak.com/istanbul-eyupsultan-gokturk-merkez-kiralik/daire/1202-97040

📄 251. ilan işleniyor: https://www.hepsiemlak.com/istanbul-maltepe-cevizli-kiralik/daire/51556-4144

📄 252. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sisli-gulbahar-kiralik/daire/35783-15215

📄 253. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sisli-gulbahar-kiralik/daire/35783-15242

📄 254. ilan işleniyor: https://www.hepsiemlak.com/istanbul-bagcilar-100-yil-kiralik/daire/130762-81

📄 255. ilan işleniyor: https://www.hepsiemlak.com/istanbul-bahcelievler-fevzi-cakmak-kiralik/daire/130762-90

📄 256. ilan işleniyor: https://www.hepsiemlak.com/istanbul-sisli-fulya-kiralik/residence/139102-86

📄 257. ilan işleniyor: https://www.hepsiemlak.com/istanbul-kagithane-ortabayir-kiralik/daire/65917-736

📄 258. ilan işleniyor: https://www.hepsiemlak.com/istanbul-avcilar-cihangir-kiralik/daire/118377-214

📄 259. ilan işleniyor: https://www.hepsiemlak.com/istanbul-kagithane

🛑 CAPTCHA varsa (İLAN) çöz ve Enter'a bas... 



📄 260. ilan işleniyor: https://www.hepsiemlak.com/istanbul-uskudar-selimiye-kiralik/daire/19061-941

📄 261. ilan işleniyor: https://www.hepsiemlak.com/istanbul-uskudar-salacak-kiralik/daire/19061-937

📄 262. ilan işleniyor: https://www.hepsiemlak.com/istanbul-uskudar-salacak-kiralik/daire/19061-908

📄 263. ilan işleniyor: https://www.hepsiemlak.com/istanbul-besiktas-levent-kiralik/daire/137955-12

📥 Veriler eklendi: istanbulkiralik.csv


IndexError: list index out of range